This notebook contains a few examples for bcc Fe to illustrate how pyVASP works

- Note: They only show how to create the inputs (set_calculation()), to submit the job, use:
   
   self.run_vasp() or self.submit_job()

In [1]:
from subprocess import run
import numpy as np
from pyVASP.code.main import pyVASP

import pandas as pd
import ase
from ase.spacegroup import crystal

# Define a custom formatting function for vectors in pandas
format_vector = lambda vector: '[' + ', '.join(['{:.10f}'.format(x) for x in vector]) + ']'

import nglview

In [2]:
# set executable
executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"

# set calculation

set crystal structure

In [3]:
a = 4.940
c = 3.493
alpha = 90
structure = crystal(('Gd', 'Co', 'Co'),
                       basis=[(0.00, 0.00, 0.00), (0.00, 0.50, 0.50), (1/3, 2/3, 0.00), (0.50, 0.50, 0.50), (0.50, 0.00, 0.50), (2/3, 1/3, 0.00)],
                       spacegroup=191,
                       cellpar=[a, a, c, alpha, alpha, alpha])

/home/emendive/.local/lib/python3.9/site-packages/ase/spacegroup/spacegroup.py:433: UserWarning: scaled_positions 1 and 3 are equivalent
  warnings.warn('scaled_positions %d and %d '
/home/emendive/.local/lib/python3.9/site-packages/ase/spacegroup/spacegroup.py:433: UserWarning: scaled_positions 1 and 4 are equivalent
  warnings.warn('scaled_positions %d and %d '
/home/emendive/.local/lib/python3.9/site-packages/ase/spacegroup/spacegroup.py:433: UserWarning: scaled_positions 2 and 5 are equivalent
  warnings.warn('scaled_positions %d and %d '


set calculation

In [4]:
# create vasp class
vasp = pyVASP(verbose="low", executable_path=executable_path, potential_path=potential_path, seed_mag=23)

In [5]:
# set magdirs for a triangular AFM state
value_of_m = 0.99

number_of_atoms = len(structure)
magdirs   = []
ms    = []
# B_CONSTRs = []
for i in range(number_of_atoms):
   if 'Co' == structure.get_chemical_symbols()[i]:
      magdirs.append([0.0, 0.0, 1.0])
      ms.append(value_of_m)
      # B_CONSTRs.append([0.0, 0.0, 0.0])
   else:
      magdirs.append([0.0, 0.0, -1.0])
      ms.append(value_of_m)
      # B_CONSTRs.append([0.0, 0.0, 0.0])

structure.new_array("magdirs", magdirs, dtype=float)
structure.new_array("ms", ms, dtype=float)
# structure.new_array("B_CONSTR", B_CONSTR, dtype=float)

In [6]:
# first prepare bfields (constraints)
vasp.prepare_bfields(I_CONSTRAINED_M="4", LAMBDA="1")
# set calculation, which creates all input files
vasp.set_calculation(structure)

visualize structure

In [7]:
view = nglview.show_ase(structure)
view.add_unitcell()
view

NGLWidget()

visualize some inputs

In [8]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
1,Co,"(0.0, 2.47, 1.7465)","(0.0, 0.0, 1.0)",0.99,100.0,"(-0.034300783435359715, 0.16617981581968974, 0...","(0.0, 0.0, 0.0)"
4,Co,"(1.6466666666666667, 3.2933333333333334, 0.0)","(0.0, 0.0, 1.0)",0.99,100.0,"(-0.11775258192804533, 0.07274014406325732, 0....","(0.0, 0.0, 0.0)"
3,Co,"(2.47, 0.0, 1.7465)","(0.0, 0.0, 1.0)",0.99,100.0,"(-0.047164822714184144, 0.03781414940626387, 0...","(0.0, 0.0, 0.0)"
2,Co,"(2.47, 2.47, 1.7465)","(0.0, 0.0, 1.0)",0.99,100.0,"(-0.02755534726913883, -0.06504238663703146, 0...","(0.0, 0.0, 0.0)"
5,Co,"(3.293333333333334, 1.646666666666667, 0.0)","(0.0, 0.0, 1.0)",0.99,100.0,"(0.005241567294710075, -0.004678017065439355, ...","(0.0, 0.0, 0.0)"
0,Gd,"(0.0, 0.0, 0.0)","(0.0, 0.0, -1.0)",0.99,100.0,"(-0.11384657122650732, -0.039408363142266575, ...","(0.0, 0.0, 0.0)"


In [9]:
np.array(vasp.df["positions"].tolist()) /a

array([[0.        , 0.5       , 0.35354251],
       [0.33333333, 0.66666667, 0.        ],
       [0.5       , 0.        , 0.35354251],
       [0.5       , 0.5       , 0.35354251],
       [0.66666667, 0.33333333, 0.        ],
       [0.        , 0.        , 0.        ]])

In [10]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

1      [-0.0343007834, 0.1661798158, 0.9854987190]
4      [-0.1177525819, 0.0727401441, 0.9903752829]
3      [-0.0471648227, 0.0378141494, 0.9981711124]
2     [-0.0275553473, -0.0650423866, 0.9975019753]
5      [0.0052415673, -0.0046780171, 0.9999753208]
0    [-0.1138465712, -0.0394083631, -0.9927164445]
Name: magmoms, dtype: object

In [11]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 12 12 12

lattice vectors =
 [[4.94  0.    0.   ]
 [0.    4.94  0.   ]
 [0.    0.    3.493]]

species = ['Co', 'Gd']

elements = 1    Co
4    Co
3    Co
2    Co
5    Co
0    Gd
Name: elements, dtype: object
